### 0. Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid

from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import pywt
import copy
import json

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/WaveletFrequencyDecomposed_CNN_Transformer/data/train_data.csv')
df.set_index('timestamp', inplace=True)

### 1. Data Preprocessing

##### 1.1 Wavelet Frequency Decompose Correlation

In [ ]:
def wavelet_decomposed_corr(df, input_window_width=30, label_window_width=10, wavelet='db4', level=3):
    X, Y = [], []
    data = df.values

    for t in range(input_window_width, len(df)-label_window_width+1):
        window_data = data[t-input_window_width : t]

        low_band, mid_band, high_band = [], [], []
        for i in range(window_data.shape[1]):
            comod_per_window = window_data[:, i]
            coeffs = pywt.wavedec(comod_per_window, wavelet, level=level)
            cA3, cD3, _, cD1 = coeffs
            low_band.append(cA3)
            mid_band.append(cD3)
            high_band.append(cD1)

        corr_low = np.corrcoef(low_band)
        corr_mid = np.corrcoef(mid_band)
        corr_high = np.corrcoef(high_band)

        corr_tensor = torch.tensor(np.stack([corr_low, corr_mid, corr_high]), dtype=torch.float32) #3x8x8
        X.append(corr_tensor)

        label_window = data[t : t+label_window_width]
        corr_next = np.corrcoef(label_window.T)
        Y.append(torch.tensor(corr_next, dtype=torch.float32))


    return torch.stack(X), torch.stack(Y)

In [ ]:
X_tensor, Y_tensor = wavelet_decomposed_corr(df)

total_size = len(X_tensor)
train_size = int(total_size * 0.8)
val_size   = int(total_size * 0.1)

test_size  = total_size - train_size - val_size

X_train = X_tensor[:train_size]
Y_train = Y_tensor[:train_size]

X_val = X_tensor[train_size:train_size + val_size]
Y_val = Y_tensor[train_size:train_size + val_size]

X_test = X_tensor[train_size + val_size:]
Y_test = Y_tensor[train_size + val_size:]

train_ds = TensorDataset(X_train, Y_train)
val_ds   = TensorDataset(X_val, Y_val)
test_ds  = TensorDataset(X_test, Y_test)

/usr/local/lib/python3.11/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


### 2. Modeling

##### 2.1 Model Structure Setting

In [ ]:
class CorrPredictorCNN(nn.Module):
    def __init__(
            self,
            num_channels=3,
            conv_channels=32,
            kernel_size = 3,
            d_model=128
            ):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(num_channels, d_model, kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.BatchNorm2d(d_model),

            nn.Conv2d(d_model, 64, kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 1, kernel_size, padding=kernel_size // 2),
        )

    def forward(self, x):
        x = self.cnn(x)
        x = x.squeeze(1)

        return x

##### 2.2 Training

In [ ]:
def train_model(model, train_loader, val_loader, optimizer_name='Adam', lr=5e-4, epochs=70, device='cuda'):
    model.to(device)

    # Optimizer 선택
    if optimizer_name == 'Adam':
        opt = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'RMSprop':
        opt = torch.optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_name == 'AdamW':
        opt = torch.optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    # Loss & LR Scheduler
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode='min', factor=0.5, patience=5, verbose=True
    )

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()

            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            opt.step()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_loss += criterion(model(xb), yb).item()
        val_loss /= len(val_loader)

        # 스케줄러 적용
        scheduler.step(val_loss)

    return val_loss

In [ ]:
model_save_path = '/content/drive/MyDrive/WaveletFrequencyDecomposed_CNN_Transformer/best_model'

In [ ]:
# Grid Search
param_grid = {
    'kernel_size': [3, 5],
    'd_model': [32, 64, 128],
    'lr': [0.001, 5e-4],
    'optimizer': ['Adam', 'RMSprop', 'AdamW'],
    'batch_size': [64, 128, 256, 512]
}

best_loss = float('inf')
best_config = None
best_model = None

for config in tqdm(ParameterGrid(param_grid)):
    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=False)
    val_loader   = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=False)

    model = CorrPredictorCNN(
        kernel_size=config['kernel_size'],
        d_model=config['d_model'],
    )
    loss = train_model(model, train_loader, val_loader,
                       optimizer_name=config['optimizer'],
                       lr=config['lr'], device=device)

    print(f"Config: {config}, Loss: {loss:.4f}")
    if loss < best_loss:
        best_loss = loss
        best_config = config

        torch.save(model.state_dict(), f"{model_save_path}/best_model_window10per30_WC_weights.pt")
        with open(f'{model_save_path}/best_model_window10per30_WC_config.json', 'w') as f:
            json.dump(best_config, f, indent=4)

# 최종 결과
print(f"\n✅ Best Config: {best_config}")
print(f"✅ Best Loss: {best_loss:.4f}")

  1%|          | 1/144 [00:05<13:02,  5.47s/it]

Config: {'batch_size': 64, 'd_model': 32, 'kernel_size': 3, 'lr': 0.001, 'optimizer': 'Adam'}, Loss: 0.0698


  1%|▏         | 2/144 [00:10<12:48,  5.41s/it]

Config: {'batch_size': 64, 'd_model': 32, 'kernel_size': 3, 'lr': 0.001, 'optimizer': 'RMSprop'}, Loss: 0.0758


  2%|▏         | 3/144 [00:16<12:47,  5.44s/it]

...

Config: {'batch_size': 512, 'd_model': 128, 'kernel_size': 5, 'lr': 0.0005, 'optimizer': 'Adam'}, Loss: 0.0703


 99%|█████████▉| 143/144 [08:13<00:03,  3.56s/it]

Config: {'batch_size': 512, 'd_model': 128, 'kernel_size': 5, 'lr': 0.0005, 'optimizer': 'RMSprop'}, Loss: 0.0685


100%|██████████| 144/144 [08:17<00:00,  3.45s/it]

Config: {'batch_size': 512, 'd_model': 128, 'kernel_size': 5, 'lr': 0.0005, 'optimizer': 'AdamW'}, Loss: 0.0682

✅ Best Config: {'batch_size': 512, 'd_model': 128, 'kernel_size': 5, 'lr': 0.001, 'optimizer': 'RMSprop'}
✅ Best Loss: 0.0661


##### 2.3 Test

In [ ]:
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    model.to(device)

    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            preds.append(pred)
            targets.append(yb)

    # (B*T, 8, 8) 텐서 형태로 합치기
    preds_tensor = torch.cat(preds, dim=0)
    targets_tensor = torch.cat(targets, dim=0)

    return preds_tensor, targets_tensor

with open(f'{model_save_path}/best_model_window10per30_WC_config.json', 'r') as f:
    best_config = json.load(f)

best_model = CorrPredictorCNN(
    kernel_size=best_config['kernel_size'],
    d_model=best_config['d_model'],
)
best_model.load_state_dict(torch.load(f"{model_save_path}/best_model_window10per30_WC_weights.pth"))

test_loader = DataLoader(test_ds, batch_size=best_config['batch_size'], shuffle=False)
preds_tensor, targets_tensor = evaluate_model(best_model, test_loader, device=device)

# 저장
torch.save({
    'preds': preds_tensor,
    'targets': targets_tensor
}, f"{model_save_path}/best_model_window10per30_WC_result.pt")

In [ ]:
# Performance metrics

preds_flat = preds_tensor.view(preds_tensor.size(0), -1).cpu().numpy()
targets_flat = targets_tensor.view(targets_tensor.size(0), -1).cpu().numpy()

mse = mean_squared_error(targets_flat, preds_flat)
mae = mean_absolute_error(targets_flat, preds_flat)
rmse = np.sqrt(mse)

# frobenius_loss
cos_sim = cosine_similarity(targets_flat, preds_flat)
mean_cos_sim = np.diag(cos_sim).mean()

# frobenius_loss
diff = preds_tensor - targets_tensor
frobenius_per_sample = torch.norm(diff, p='fro', dim=(1, 2))
mean_frobenius = frobenius_per_sample.mean().item()

print(f"\n📊 Evaluation Results:")
print(f"MSE               : {mse:.5f}")
print(f"MAE               : {mae:.5f}")
print(f"RMSE              : {rmse:.5f}")
print(f"Cosine Similarity : {mean_cos_sim:.5f}")
print(f"Frobenius Norm    : {mean_frobenius:.5f}")


📊 Evaluation Results:
MSE               : 0.06904
MAE               : 0.18716
RMSE              : 0.26275
Cosine Similarity : 0.93967
Frobenius Norm    : 1.91697
